# **MÓDULO 21 - Projeto de Credit Score - Árvore de Decisão**


No módulo 17, vocês realizaram a primeira etapa do projeto de crédito de vocês. Então fizeram o tratamendo dos dados, balancearam as classes, transformaram as variáveis categóricas e separam base de treino e teste. Já no módulo 14, aplicaram a base já tratada o algoritmo de Naive Bayes, onde avaliaram os resultados das previsões. Nesse módulo aplicaremos a nossa base o algoritmo da árvore de decisão.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1) Comece carregando as bases de treino (X e y) e teste (X e y).
Verifique se o número de linhas condiz, se as variáveis estão corretas sendo apenas a de score para y e as demais nas bases de X e por último, se Y está balanceada no teste.

In [ ]:
# Leitura dos dados .csv
credit_data = pd.read_csv('CREDIT_SCORE_M20.csv')

# Apagando a coluna descenessária 'Unnamed: 0'
credit_data = credit_data.drop('Unnamed: 0', axis=1)

# Visualização dos resultados
credit_data.head()

,Age,Income,Number of Children,Gender_encoded,ED__BACHELOR'S DEGREE,ED__DOCTORATE,ED__HIGH SCHOOL DIPLOMA,ED__MASTER'S DEGREE,Marital Status Code,Home Ownership Code,Credit Score Code
0,25,50000.0,0,0,True,False,False,False,1,1,1
1,30,100000.0,2,1,False,False,False,True,0,0,1
2,35,75000.0,1,0,False,True,False,False,0,0,1
3,40,125000.0,0,1,False,False,True,False,1,0,1
4,45,100000.0,3,0,True,False,False,False,0,0,1


Separando as bases do arquivo CSV

In [44]:
from sklearn.model_selection import train_test_split

X = credit_data.drop('Credit Score Code', axis=1)
y = credit_data['Credit Score Code']

Aplicando a separação das bases de treino e de teste

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0
)

Verificando o tamanho das bases de dados

In [49]:
print('O tamanho de X_train é: ', X_train.value_counts().sum())
print('O tamanho de X_test é: ', X_test.value_counts().sum())
print('O tamanho de y_train é: ', y_train.value_counts().sum())
print('O tamanho de y_test é: ', y_test.value_counts().sum())

O tamanho de X_train é:  123
O tamanho de X_test é:  41
O tamanho de y_train é:  123
O tamanho de y_test é:  41


Verificando o balanceamento da base de dados

In [57]:
print('Verificando o balanceamento de dados da base treino')
print(y_train.value_counts(normalize=True) * 100)

print('\n\nVerificando o balanceamento de dados da base teste')
print(y_test.value_counts(normalize=True) * 100)

Verificando o balanceamento de dados da base treino
Credit Score Code
1    66.666667
0    22.764228
2    10.569106
Name: proportion, dtype: float64


Verificando o balanceamento de dados da base teste
Credit Score Code
1    75.609756
0    19.512195
2     4.878049
Name: proportion, dtype: float64


A base de dados está desbalanceada. No exercício anterior conforme trabalhado, temos que:  
1: Alto  
0: Médio  
2: Baixo  

Portanto, a maior parte dos nossos dados são de Credit Score Alto. Vamos balancear **apenas na base de treino** para não enviesar nosso modelo e aplicar o teste com os dados reais.

In [62]:
from imblearn.over_sampling import SMOTE

# Cria uma instância smote
smote = SMOTE(random_state=1)

# Aplicando apenas na base de TREINO
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificando os tamanhos da base
print('Balanceamento em y_train: \n')
print(y_train_balanced.value_counts(normalize=True))

print('\nTamanho de X_train: ', X_train_balanced.value_counts().sum())
print('\nTamanho de y_train: ', y_train_balanced.value_counts().sum())

Balanceamento em y_train: 

Credit Score Code
1    0.333333
0    0.333333
2    0.333333
Name: proportion, dtype: float64

Tamanho de X_train:  246

Tamanho de y_train:  246


# 2) Explique com suas palavras, qual o passo a passo para a aplicação do algoritmo da árvore de decisão, não esqueça de citar a etapa de avaliação do modelo e também como podemos melhorar nosso modelo.

In [ ]:
#Sua resposta aqui

# 3) Aplique o algortimo da árvore de decisão aos dados de treinamento, utilizando critério de Gini e random state = 0.
Traga a acurácia para o modedlo com os dados de treino.

In [ ]:
#seu código aqui

# 4) Aplique o modelo aos dados de teste e realize a avaliação dos resultados. Não se esqueça de avaliar com as suas palavras e comparar o desempenho da base treino com a teste.

In [ ]:
#seu código aqui

# 5) Plote a árvore de decisão.
É possível fazer uma avaliação visual? Qual a profundidade da árvore?

In [ ]:
#seu código aqui

# 6) Identifique as 2 principais features do modelo.


In [ ]:
#seu código aqui

# 7) Rode um modelo de árvore de decisão apenas com as 2 principais features encontradas. E avalie os resultados. Para você o desempenho da árvore está melhor que o modelo anterior? Justifique.

In [ ]:
#seu código aqui

# 8) Compare os resultados obtidos com a árvore de decisão com os resultados do Naive Bayes (Exercício módulo 20). Qual parece ter se adequado melhor aos dados e tem melhores resultados de avaliação? Justifique.

In [ ]:
#Seu texto aqui